In [8]:
pip install requests beautifulsoup4 tqdm selenium  

ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random
import re
import os
from tqdm import tqdm

BASE_URL = "https://law.stackexchange.com"
TAG_URL = BASE_URL + "/questions?tab=votes&pagesize=50"

def setup_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    return webdriver.Chrome(options=options)

def wait_for_element(driver, css_selector, timeout=10):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
        )
    except Exception as e:
        print(f"[!] Timeout waiting for element: {css_selector} -> {str(e)}")

def get_question_links(driver, page=1):
    url = TAG_URL + f"&page={page}"
    driver.get(url)
    wait_for_element(driver, "div.s-post-summary")

    soup = BeautifulSoup(driver.page_source, "html.parser")
    question_links = []
    for q_summary in soup.select("div.s-post-summary"):
        link_tag = q_summary.select_one("a.s-link")
        if link_tag and "/questions/" in link_tag["href"]:
            full_url = BASE_URL + link_tag["href"].split("?")[0]
            question_links.append(full_url)
    return list(set(question_links))

def scrape_question_page(driver, url):
    driver.get(url)
    wait_for_element(driver, "h1.fs-headline1")

    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        title_tag = soup.select_one("a.question-hyperlink, h1.fs-headline1")
        if not title_tag:
            return None
        prompt = title_tag.get_text(strip=True)

        # Chỉ lấy những tiêu đề là câu hỏi
        if not prompt.endswith("?"):
            return None

        answers = []
        for ans in soup.select("div.answer"):
            vote_tag = ans.select_one("div.js-vote-count")
            body_tag = ans.select_one("div.js-post-body")

            if not vote_tag or not body_tag:
                continue

            try:
                vote_count = int(vote_tag.get_text(strip=True))
            except ValueError:
                vote_count = 0

            text = body_tag.get_text(separator=" ", strip=True)
            if len(text) > 50:
                answers.append((vote_count, text))

        if answers:
            best_answer = max(answers, key=lambda x: x[0])[1]
            return {
                "prompt": prompt,
                "response": best_answer
            }

    except Exception as e:
        print("Error scraping:", url, "->", str(e))
    return None

def scrape_all(pages, delay):
    driver = setup_driver()
    dataset = []

    try:
        for page in range(1, pages + 1):
            print(f"Scraping page {page}...")
            links = get_question_links(driver, page)
            for link in tqdm(links):
                data = scrape_question_page(driver, link)
                if data:
                    dataset.append(data)
                time.sleep(delay + random.uniform(0, 1.5))
    finally:
        driver.quit()

    return dataset

def save_jsonl(dataset, out_file="law_stackexchange_data.jsonl"):
    with open(out_file, "w", encoding="utf-8") as f:
        for item in dataset:
            f.write(json.dumps({
                "prompt": clean_text(item["prompt"]),
                "response": clean_text(item["response"])
            }, ensure_ascii=False) + "\n")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    text = re.sub(r"\s+", " ", text)
    text = text.strip().strip('"').strip("'")
    return text.strip()


In [10]:
if __name__ == "__main__":
    data = scrape_all(pages=120, delay=0.5)
    save_jsonl(data)
    print(f"Collected and cleaned {len(data)} Q&A pairs. Saved to 'law_stackexchange_data.jsonl'")

Scraping page 1...


 60%|██████    | 30/50 [00:57<00:34,  1.74s/it]

[!] Timeout waiting for element: h1.fs-headline1 -> Message: 
Stacktrace:
#0 0x59b94523329a <unknown>
#1 0x59b944c9f040 <unknown>
#2 0x59b944cf0990 <unknown>
#3 0x59b944cf0b81 <unknown>
#4 0x59b944d3f664 <unknown>
#5 0x59b944d166bd <unknown>
#6 0x59b944d3cace <unknown>
#7 0x59b944d16463 <unknown>
#8 0x59b944ce2c55 <unknown>
#9 0x59b944ce38c1 <unknown>
#10 0x59b9451f81bb <unknown>
#11 0x59b9451fc0c9 <unknown>
#12 0x59b9451e0a39 <unknown>
#13 0x59b9451fcc78 <unknown>
#14 0x59b9451c56bf <unknown>
#15 0x59b945220948 <unknown>
#16 0x59b945220b26 <unknown>
#17 0x59b945232176 <unknown>
#18 0x7a9008f84ac3 <unknown>



100%|██████████| 50/50 [01:45<00:00,  2.12s/it]


Scraping page 2...


...

[!] Timeout waiting for element: h1.fs-headline1 -> Message: 
Stacktrace:
#0 0x59b94523329a <unknown>
#1 0x59b944c9f040 <unknown>
#2 0x59b944cf0990 <unknown>
#3 0x59b944cf0b81 <unknown>
#4 0x59b944d3f664 <unknown>
#5 0x59b944d166bd <unknown>
#6 0x59b944d3cace <unknown>
#7 0x59b944d16463 <unknown>
#8 0x59b944ce2c55 <unknown>
#9 0x59b944ce38c1 <unknown>
#10 0x59b9451f81bb <unknown>
#11 0x59b9451fc0c9 <unknown>
#12 0x59b9451e0a39 <unknown>
#13 0x59b9451fcc78 <unknown>
#14 0x59b9451c56bf <unknown>
#15 0x59b945220948 <unknown>
#16 0x59b945220b26 <unknown>
#17 0x59b945232176 <unknown>
#18 0x7a9008f84ac3 <unknown>



100%|██████████| 50/50 [01:35<00:00,  1.92s/it]


Scraping page 120...


100%|██████████| 50/50 [01:34<00:00,  1.90s/it]


Collected and cleaned 4878 Q&A pairs. Saved to 'law_stackexchange_data.jsonl'
